# Mathematical Statistics Homework Assignment
#### 2023.05
#### Richárd Kiss - KAYXFT

# 1. Introduction

In this assignment I use a dataset that was generated as part of my PhD research. The dataset contains music playlists and song recommendations for these playlists generated with two different algorithms. In this assignment I am performing statistical analysis to compare these two algorithms in terms of recommendation performance.

## The two recommendation algorithms
### Algorithm 1.
The first algorithm recommends songs based solely on their popularity. It consistently returns the top $K$ most popular songs, irrespective of the particular songs present in the playlist for which we are making new song recommendations. Despite its simplicity, this approach is very effective and often used as a benchmark in the evaluation of recommendation systems.

### Algorithm 2.
The second algorithm employs Skip-Gram with negative sampling to learn insightful, low-dimensional song representations (denoted by $w_i \in \mathbb{R}^l$ for all songs $i$, where $l$ is the dimensionality of the representation). The core concept revolves around discerning whether song pairs originate from the original data distribution, $P_{orig}$, or from a synthetic noise distribution, $P_{noise}$.

The song pairs in the original distribution are generated by sampling songs from a playlist within a specific context window (of size $c$), and corresponding noise samples are created by randomly pairing songs that do not appear together within a context window (of the same size $c$).

To reduce computational demands, we ease the restriction of never appearing together. Since the co-appearance graph of songs is usually sparse, choosing two random songs seldom results in a pair that co-appears in the dataset. Although there's still a slim chance, this compromise generally speeds up the sampling process without drastically affecting the results. (You can find more details about this method  [here](https://www.baeldung.com/cs/nlps-word2vec-negative-sampling)).

Once we've learned the song representations $w$, we can use them to generate playlist recommendations. The process begins by selecting the $K$ most similar songs for each song in the playlist. Then, we count the number of times a particular song appears within the $K$ proximity of each song in the playlist and rank them in descending order. Finally, we return the top $K$ songs from this sorted list.


## The data
spotify dataset, #>30, some basic statistics (num songs, albums, artists etc.)


# 2. Statistical analysis
## Hypotheses
__First Hypothesis__

I want to check whether the frequencies of songs (number of appearances in playlists) comes from an exponential distribution. To test this I first fit an exponential distribution on the frequencies then I perform a one sample Kolmogorov-Smirnov test with the fitted exponential.

__Second Hypothesis__

Recommendations generated with Algorithm 2. have higher recall ($\frac{TP}{TP+FN}$) on average than those generated with Algorithm 1. To test this I am performing a two sample T-test on the recall values.

__Regression analysis__

...
## The data I use in this notebook

In [1]:
import os
import json
import numpy as np
import scipy.stats as stats

In [2]:
DATA_FOLDER = "../data/"

In [3]:
def load(*path):
    with open(os.path.join(*path)) as f:
        return json.load(f)

idx2song = load(DATA_FOLDER, "metadata", "idx2song.json")
freqs = load(DATA_FOLDER, "metadata", "frequencies.json")
# it is necessary to remap song id-s to song names in the freqs json
freqs = {idx2song[k]: v for k,v in freqs.items()}
algo1 = load(DATA_FOLDER, "popularity.json")
algo2 = load(DATA_FOLDER, "euclidean.json")
qrels = load(DATA_FOLDER, "qrels.json")

In [17]:
def recall(tp, fn):
    return tp / (tp + fn)

def get_recalls(qrels, recs):
    for (q_id, q_data), (r_id, r_data) in zip(qrels.items(), recs.items()):
        assert q_id == r_id, f"ID mismatch {q_id} {r_id}"
        actual = set(q_data.keys())
        predicted = set(r_data.keys())
        tp = len(actual & predicted)
        fn = len(actual.difference(predicted))
        yield recall(tp, fn)

In [23]:
recalls1 = list(get_recalls(qrels, algo1))
recalls2 = list(get_recalls(qrels, algo2))

In [25]:
stats.ttest_rel(recalls1, recalls2)

TtestResult(statistic=-246.58068655214413, pvalue=0.0, df=84791)